In [2]:
import openai
from langchain.chains import LLMChain
import os
from dotenv import load_dotenv

In [3]:
# 加載 .env 檔案
load_dotenv()

# 從環境變數中獲取 API 金鑰
openai.api_key = os.getenv('OPENAI_API_KEY')

In [4]:
# 初始化 Langchain
chain = LLMChain()

ValidationError: 2 validation errors for LLMChain
prompt
  field required (type=value_error.missing)
llm
  field required (type=value_error.missing)